# Making a New Cancer Database

Taking the data from the federal government, specifically the [United States Cancer Statistics](https://www.cdc.gov/cancer/uscs/dataviz/download_data.htm). Looking specifically at the BYAREA.TXT file from the 1999-2017 zip file, which gives us the number of occurances of each cancer along with crude and age adjusted rates. This data set futher breaks down by sex and race while giving information of  location of cancer, and sex by year.

## Initial processing

Change their NaN/None ~/+ into np.nan, also want to cut down on file size by dropping the less interesting columns and decreasing the number of years.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('BYAREA.txt',sep="|",low_memory=False) #note need this file to run, too large to upload to github
df = df.replace(['~','+'],np.nan)

In [2]:
# Lets delete the less interesting data
df = df.drop(columns=["AGE_ADJUSTED_CI_LOWER","AGE_ADJUSTED_CI_UPPER","CRUDE_CI_LOWER","CRUDE_CI_UPPER"])

In [3]:
# Also, this file is so large, what happens if we restrict to five year intervals? (2019, 201)
# first lets delete any ranges by changing them to 0
df.YEAR= df.YEAR.apply(lambda x: np.where(x.isdigit(),x,0))
df.YEAR = df.YEAR.astype("int64")

In [4]:
# Lets look at what years we have the most data from
df.groupby('YEAR').count()

,AREA,AGE_ADJUSTED_RATE,COUNT,EVENT_TYPE,POPULATION,RACE,SEX,SITE,CRUDE_RATE
YEAR,,,,,,,,,
0,57594,57594,40113,57594,57594,57594,57594,57594,57594
1999,55374,55303,26449,55374,55374,55374,55374,55374,55303
2000,55670,55594,26955,55670,55670,55670,55670,55670,55594
2001,56558,56558,27309,56558,56558,56558,56558,56558,56558
2002,56558,56558,27467,56558,56558,56558,56558,56558,56558
2003,57594,57594,28385,57594,57594,57594,57594,57594,57594
2004,57594,57594,28512,57594,57594,57594,57594,57594,57594
2005,57594,57594,28794,57594,57594,57594,57594,57594,57594
2006,57594,57594,29017,57594,57594,57594,57594,57594,57594


In [5]:
#have 57594 entries for each year, will delete all but 2017, 2012, 2007, 2002 to shrink the data set but give some yearly change
df_lessYears = df[(df['YEAR'] ==2002) | (df['YEAR'] ==2007) | (df['YEAR'] ==2012) | (df['YEAR'] ==2017)]

## Combining Mortality Data
Think to make this dataset more useful, the mortality events should be in the same rpw as the corresponding incident row.

To do so, will break apart the rows with mortalities and seperate the important data and recombine.

In [6]:
df_mortality = df_lessYears[df_lessYears['EVENT_TYPE']=='Mortality'] # seperate the mortality rows
df_removeMortRows = df_lessYears.drop(df_lessYears[df_lessYears['EVENT_TYPE']=='Mortality'].index) # drop the event column as no longer needed

In [7]:
df_removeMortRows = df_removeMortRows.drop(columns=['EVENT_TYPE']) #Drop event column from indicendt data too
# rename columns for clarity
df_removeMortRows.columns = ["Area","Incidence.Age_Adjust_Rate","Incidence.Count","Population","Race","Sex","Site","Year","Incidence.Crude_Rate"] 

In [8]:
df_mortality.index = df_mortality.index -1 # fix indicies to line up, assumes mortality always on row below!

In [9]:
# Drop redundant columns in the mortality dataset and rename remaining
df_uniqueMortality = df_mortality.drop(columns=["AREA","POPULATION","RACE","SEX","YEAR","SITE","EVENT_TYPE"])
df_uniqueMortality.columns = ["Mortality.Age_Adjust_Rate","Mortality.Count","Mortality.Crude_Rate"]

In [10]:
# recombine with a concatination
df_recombine =  pd.concat([df_removeMortRows,df_uniqueMortality.reindex(df_removeMortRows.index)],axis=1)

## Final Processing

I am relatively happy with this data. Might be less helpful as it will require sorting by the user as I did not transpose each cancer/race/sex to its own column as I am not aware of an efficient way to do that. Instead boolean sorting can be done to get the data wanted by the each student.

Think I will make the meta data file by hand. Took icon from [wikipedia](https://commons.wikimedia.org/wiki/File:Multiple_myeloma_(1)_MG_stain.jpg).

Therefore can rearrange the columns to a form which makes sense and then export to csv as final product (at least for this iteration):

In [11]:
# Making the meta data file
cols = df_recombine.columns.tolist()
cols = [cols[0],cols[7],cols[3],cols[6],cols[4],cols[5],cols[2],cols[10],cols[8],cols[11],cols[1],cols[9]]
cols

['Area',
 'Year',
 'Population',
 'Site',
 'Race',
 'Sex',
 'Incidence.Count',
 'Mortality.Count',
 'Incidence.Crude_Rate',
 'Mortality.Crude_Rate',
 'Incidence.Age_Adjust_Rate',
 'Mortality.Age_Adjust_Rate']

In [12]:
df_ordered = df_recombine[cols]
df_ordered.to_csv('cancer_data.csv',header=False,index=False)

PermissionError: [Errno 13] Permission denied: 'cancer_data.csv'

In [13]:
df_ordered

,Area,Year,Population,Site,Race,Sex,Incidence.Count,Mortality.Count,Incidence.Crude_Rate,Mortality.Crude_Rate,Incidence.Age_Adjust_Rate,Mortality.Age_Adjust_Rate
6,Alabama,2002,2314370,All Cancer Sites Combined,All Races,Female,10151,4407,438.6,190.4,389.9,163.1
16,Alabama,2007,2407275,All Cancer Sites Combined,All Races,Female,11012,4492,457.4,186.6,396.1,156.4
26,Alabama,2012,2479787,All Cancer Sites Combined,All Races,Female,11921,4568,480.7,184.2,397.9,147.5
38,Alabama,2017,2516043,All Cancer Sites Combined,All Races,Female,12661,4689,503.2,186.4,397.4,138.5
46,Alabama,2002,2314370,Brain and Other Nervous System,All Races,Female,139,100,6.0,4.3,5.6,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...
1145622,Wyoming,2017,544321,Thyroid,White,Male and Female,89,NaN,16.4,0.2,15.6,0.1
1145630,Wyoming,2002,477938,Urinary Bladder,White,Male and Female,129,16,27.0,3.3,26.6,3.5
1145640,Wyoming,2007,508757,Urinary Bladder,White,Male and Female,122,20,24.0,3.9,22.9,3.8
1145650,Wyoming,2012,543563,Urinary Bladder,White,Male and Female,153,28,28.1,5.2,25.2,4.9
